In [2]:
import geemap
import ee
ee.Authenticate()

In [3]:
ee.Initialize(project= "ee-shirwadkarsejal809")

In [4]:
m = geemap.Map()
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [5]:
India = ee.FeatureCollection("projects/ee-shirwadkarsejal809/assets/India_state")
m.addLayer(India, {}, "India")
m

Map(bottom=812.0, center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

In [15]:
Ganga = ee.FeatureCollection("projects/ee-shirwadkarsejal809/assets/Ganga")
m.addLayer(Ganga, {}, "Ganga")
m

Map(bottom=3902.0, center=[21.22794190505816, 81.80655045186813], controls=(WidgetControl(options=['position',…

In [13]:
buffer_distance = 50000
Ganga_buffer = Ganga.map(lambda f: f.buffer(buffer_distance))
m.addLayer(Ganga_buffer, {}, "Ganga_buffer")
m

Map(bottom=1829.0, center=[41.44272637767212, 73.26665859165514], controls=(WidgetControl(options=['position',…

In [16]:
NDWI = ee.ImageCollection("LANDSAT/COMPOSITES/C02/T1_L2_8DAY_NDWI").select('NDWI')

In [17]:
NDWI_Ganga = NDWI.map(lambda img: img.set('doy',ee.Date(img.get('system:time_start')).getRelative('day', 'year')));
distinctDOY = NDWI_Ganga.filterDate('2022-01-01', '2023-12-31');

In [19]:
filter = ee.Filter.equals(leftField='doy', rightField ='doy')
join = ee.Join.saveAll('doy_matches')
joincol = ee.ImageCollection(join.apply(distinctDOY, NDWI_Ganga, filter))

In [20]:
def add_date_from(img):
  doy = ee.Number(img.get('doy'))

In [21]:
composite = joincol.map(lambda img: ee.ImageCollection.fromImages(img.get('doy_matches')).reduce(ee.Reducer.mean()).set('doy',ee.Number(img.get('doy'))))

In [22]:
Vis_para = {
  'min': 0.0,
  'max': 0.1,
  'palette': [
    '0000ff', '00ffff', 'ffff00', 'ff0000', 'ffffff'
  ],
};

In [23]:
rgbvis= composite.map(lambda img: img.visualize(bands = ['NDWI_mean'],**Vis_para).clip(Ganga_buffer))

In [26]:
gifParams = {
  'region': Ganga_buffer.geometry(),
  'dimensions': 600,
  'crs': 'EPSG:4326',
  'framesPerSecond': 80,
  'format':'gif',
  'addDate': 'yyyy-mm-dd'
}

In [27]:
print(rgbvis.getVideoThumbURL(gifParams))

https://earthengine.googleapis.com/v1/projects/ee-shirwadkarsejal809/videoThumbnails/2800f20de0d2f4e1e18d4f41f04df517-ae064e08843ff8112a458cb47e7416c3:getPixels
